In [44]:
%load_ext autoreload
%autoreload 2

import utilities
from evaluator import Evaluator
from embedder import Embedder
from embedder import Transformer
from embedder import Transformer_GNN
from embedder import Transformer_GNN_embedder
from embedder import Preprocessing_scaler
from Graph_generator import Graph_Generator
import tmp_generator as tmp
from visualizator import Visualizator

from eden import graph
import umap 

import numpy as np
from sklearn.decomposition import TruncatedSVD

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Generate graphs + LDP

In [142]:
n_graphs = 500

# 20 permutations
gg = Graph_Generator(n_graphs=n_graphs,
                     degree = 4)
g1, na1, ea1 = gg.generate(20,10,10,1)
g2, na2, ea2 = gg.generate(20,10,10,10)

graphs, labels = gg.generate_set(g1,g2,na1, ea1,na2, ea2,15,15,plot=False)

functions = [utilities.degree,utilities.clust_coefficient,utilities.local_degree_profile]
graphs = utilities.add_info_to_nodes(graphs,functions)


max_n_nodes, n_attributes = utilities.find_shapes(graphs)
n_classes = len(np.unique(labels))

print("done")

done


In [143]:
def tmp_preprocessing_vectorize(graphs):        

    for g in graphs:
        
        c=0
        for n in g.nodes():
            a = g.nodes[n]['vec']
            del g.nodes[n]['vec']
            g.nodes[n]['label'] = str(a[0])
            g.nodes[n]['vec'] = a
            c=c+1

    for g in graphs:
        c=0
        for n in g.edges():
            a = g.edges[n]['vec']
            del g.edges[n]['vec']      
            g.edges[n]['label'] = str(a[0])
            g.edges[n]['vec'] = a[0]
            c=c+1

    return(graphs)

graphs = tmp_preprocessing_vectorize(graphs)

In [25]:
# setting 

dim = [2,3,5]
times = 1


batch_size = 32
validation_split = 0.2
epochs_GNN = 100
epochs_autoencoder = 200
verbose=0
plot=False

## Embedder 1
### Spektral + PCA

In [21]:
my_GNN = tmp.generate_GNN(max_n_nodes,
                      n_attributes,
                      n_classes,
                      batch_size = batch_size,
                      validation_split = validation_split,
                      epochs = epochs_GNN,
                      verbose = verbose, 
                      plot=plot)

m_1 = Transformer(my_GNN)
m_2 = Transformer(TruncatedSVD())
emb_1 = Embedder([m_1,m_2],"Spektral + PCA")

## Embedder 2
### Spektral + Small 128 - n_components

In [22]:
my_GNN_small = tmp.gen_gnn_small(max_n_nodes,
                                 n_attributes,
                                 batch_size = batch_size,
                                 validation_split = validation_split,
                                 epochs = epochs_GNN,
                                 dim = dim,
                                 plot = plot)

m_1 = Transformer(my_GNN_small)
emb_2 = Embedder([m_1],"Spektral + small")

## Embedder 3
### Spektral + Dense 128 - 64 - 32 - 16 -  8 - n_components

In [23]:
my_GNN_dense = tmp.gen_gnn_dense(max_n_nodes,
                                 n_attributes,
                                 batch_size = batch_size,
                                 validation_split = validation_split,
                                 epochs = epochs_GNN,
                                 dim = dim,
                                 plot = plot)

m_1 = Transformer(my_GNN_dense)
emb_3 = Embedder([m_1],"Spektral + dense")

## Embedder 4
### Spektral fino a 128 + Autoencoder 128 - 64- 16 - 8 - 5 - 3 - 2
### Preprocessing_scaler -0.5 ; 0.5

In [24]:
# modello 1 GNN
my_GNN = tmp.generate_GNN(max_n_nodes,n_attributes,n_classes,
                           batch_size=batch_size,
                           validation_split=validation_split,
                           verbose=verbose,
                           epochs=epochs_GNN,
                           plot=plot)
m_1 = Transformer(my_GNN)

# preprocessing scaler
my_scaler = Preprocessing_scaler([-0.5, 0.5])
# modello 2 Autoencoder
my_Autoencoder = tmp.gen_transf_autoencoder(batch_size=batch_size,
                                            validation_split=validation_split,
                                            epochs=epochs_autoencoder, 
                                            verbose=verbose,
                                            scaler=my_scaler,
                                            dim = dim,
                                            plot=plot)

m_2 = Transformer(my_Autoencoder)


emb_4 = Embedder([m_1,m_2], name="Spketral 128 - Autoencoder denso")

## Embedder 5
### Vectorize + pca 1000 + umap

In [146]:
vetcoriz = graph.Vectorizer(complexity = 5)
m_1= Transformer(vetcoriz, has_fit = False)
pca = TruncatedSVD(n_components=1000)
m_2 = Transformer(pca)
uma = umap.UMAP()
m_3 = Transformer(uma)

emb_5 = Embedder([m_1,m_2,m_3], name="Vectorize-5, PCA-1000, UMAP")

In [151]:
vis_500 = utilities.repeat_n_times(graphs, labels, [emb_5,emb_4], [2,3,5], 1,test_size=0.3,seed = 11)

	 iteration n: 1
	 	 dim:  2
	 	 	  Vectorize-5, PCA-1000, UMAP
	 	 	  Spketral 128 - Autoencoder denso
0
	 	 dim:  3
	 	 	  Vectorize-5, PCA-1000, UMAP
	 	 	  Spketral 128 - Autoencoder denso
0
	 	 dim:  5
	 	 	  Vectorize-5, PCA-1000, UMAP
	 	 	  Spketral 128 - Autoencoder denso
0


In [152]:
vis_500.summary(std = False)

                                      2      3      5    mean
--------------------------------  -----  -----  -----  ------
Vectorize-5, PCA-1000, UMAP       0.805  0.806  0.873   0.828
Spketral 128 - Autoencoder denso  0.439  0.534  0.562   0.512


In [153]:
vis_500.summary()

                                      2    STD      3    STD      5    STD    mean
--------------------------------  -----  -----  -----  -----  -----  -----  ------
Vectorize-5, PCA-1000, UMAP       0.805  0.098  0.806  0.082  0.873  0.07    0.456
Spketral 128 - Autoencoder denso  0.439  0.184  0.534  0.122  0.562  0.115   0.326


# Experiments

In [26]:
emb = [emb_1,emb_2,emb_3,emb_4]

vis_500 = utilities.repeat_n_times(graphs, labels, emb, dim, times,test_size=0.3,seed = 11)

	 iteration n: 1
	 	 dim:  2
	 	 	  Spektral + PCA
reset state TRANSFORMER
reset_state GNN
reset state TRANSFORMER
0


/home/antonio/anaconda3/envs/tesi/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/antonio/anaconda3/envs/tesi/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


	 	 	  Spektral + small
reset state TRANSFORMER
reset state GNN embed
0
	 	 	  Spektral + dense
reset state TRANSFORMER
reset state GNN embed
0


/home/antonio/anaconda3/envs/tesi/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/antonio/anaconda3/envs/tesi/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


	 	 	  Spketral 128 - Autoencoder denso
reset state TRANSFORMER
reset_state GNN
reset state TRANSFORMER
reset state AUTOENCODER
0
	 	 dim:  3
	 	 	  Spektral + PCA
reset state TRANSFORMER
reset_state GNN
reset state TRANSFORMER
0
	 	 	  Spektral + small
reset state TRANSFORMER
reset state GNN embed
0
	 	 	  Spektral + dense
reset state TRANSFORMER
reset state GNN embed
0
	 	 	  Spketral 128 - Autoencoder denso
reset state TRANSFORMER
reset_state GNN
reset state TRANSFORMER
reset state AUTOENCODER
0
	 	 dim:  5
	 	 	  Spektral + PCA
reset state TRANSFORMER
reset_state GNN
reset state TRANSFORMER
0
	 	 	  Spektral + small
reset state TRANSFORMER
reset state GNN embed
0
	 	 	  Spektral + dense
reset state TRANSFORMER
reset state GNN embed
0
	 	 	  Spketral 128 - Autoencoder denso
reset state TRANSFORMER
reset_state GNN
reset state TRANSFORMER
reset state AUTOENCODER
0


In [35]:
vis_500.summary(std = False)
vis_500.rank()

                                      2      3      5    mean
--------------------------------  -----  -----  -----  ------
Spektral + PCA                    0.594  0.495  0.558   0.549
Spektral + small                  0.644  0.646  0.598   0.629
Spektral + dense                  0.533  0.565  0.476   0.525
Spketral 128 - Autoencoder denso  0.58   0.622  0.455   0.552
                                    2    3    5     mean
--------------------------------  ---  ---  ---  -------
Spektral + PCA                      2    4    2  2.66667
Spektral + small                    1    1    1  1
Spektral + dense                    4    3    3  3.33333
Spketral 128 - Autoencoder denso    3    2    4  3
